In [1]:
%matplotlib inline

# Tensores

Los tensores son una estructura de datos especializada que es muy similar a los arrays y matrices. En PyTorch, se usan tensores para codificar las entradas y salidas de un modelo, así como los parámetros del modelo.

Los tensores son similares a los ndarrays de NumPy, excepto que los tensores pueden ejecutarse en GPU u otros aceleradores de hardware. De hecho, los tensores y las matrices de NumPy a menudo pueden compartir la misma memoria subyacente, lo que elimina la necesidad de copiar datos. Los tensores también están optimizados para la diferenciación automática (Autograd).

--------------

## Carga de librerias

In [2]:
import torch
import numpy as np

--------------

## Inicialización de tensores

Los tensores se pueden inicializar de varias formas

### Directamente con datos

In [3]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

### Desde un array de NumPy


In [4]:
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

La matriz de Numpy y el Tensor comparten memoria, si se modifica uno se ve reflejado el cambio en el otro

In [5]:
np_array[0, 0] = 99
x_np[1, 1] = 100

print(f"Numpy matrix: \n {np_array} \n")
print(f"Tensor: \n {x_np} \n")

Numpy matrix: 
 [[ 99   2]
 [  3 100]] 

Tensor: 
 tensor([[ 99,   2],
        [  3, 100]], dtype=torch.int32) 



Esto es así, aunque el tensor esté en la GPU

In [6]:
gpu_device = "cuda" if torch.cuda.is_available() else "cpu"
cpu_device = "cpu"

x_np.to(gpu_device)

x_np[1, 1] = 101

print(f"Numpy matrix: \n {np_array} \n")
print(f"Tensor: \n {x_np} \n")

Numpy matrix: 
 [[ 99   2]
 [  3 101]] 

Tensor: 
 tensor([[ 99,   2],
        [  3, 101]], dtype=torch.int32) 



### Desde otro tensor

In [5]:
x_ones = torch.ones_like(x_data) # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

Ones Tensor: 
 tensor([[1, 1],
        [1, 1]]) 

Random Tensor: 
 tensor([[0.7157, 0.3992],
        [0.3843, 0.9938]]) 



### Con valores aleatorios o constnates

In [7]:
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

Random Tensor: 
 tensor([[0.9277, 0.6535, 0.7438],
        [0.0253, 0.8267, 0.8434]]) 

Ones Tensor: 
 tensor([[1., 1., 1.],
        [1., 1., 1.]]) 

Zeros Tensor: 
 tensor([[0., 0., 0.],
        [0., 0., 0.]])


--------------




## Atributos de un tensor

Se pueden obtener los atributos de un tensor, como el tamaño, el tipo y el dispositivo en el que está

In [8]:
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


--------------




## Operaciones con Tensores

Hay más de 100 operaciones con tensores, en este [enlace](https://pytorch.org/docs/stable/torch.html). Cada una de estas operaciones se puede ejecutar en la GPU (normalmente a velocidades más altas que en una CPU).

De forma predeterminada, los tensores se crean en la CPU. Necesitamos mover explícitamente los tensores a la GPU usando el método .to (después de verificar la disponibilidad de la GPU). Hay que tener en cuenta que copiar grandes tensores entre dispositivos puede resultar caro en términos de tiempo y memoria.

In [9]:
# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to('cuda')

print(f"Device tensor is stored on: {tensor.device}")

Device tensor is stored on: cuda:0


Veamos algunas de las operaciones. Al ser muy similares a Numpy, la API de Tensor es muy fácil de usar.

### Indexación y segmentación como en Numpy

In [10]:
tensor = torch.ones(4, 4)
print('First row: ', tensor[0])
print('First column: ', tensor[:, 0])
print('Last column:', tensor[..., -1])
tensor[:,1] = 0
print(tensor)

First row:  tensor([1., 1., 1., 1.])
First column:  tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


### Unión de tensores

Se puede utilizar torch.cat para concatenar una secuencia de tensores a lo largo de una dimensión determinada. También se puede usar torch.stack para unir otro tensor y es sutilmente diferente de torch.cat.

In [11]:
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


### Operaciones aritméticas

In [12]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(tensor)
mat_mult = torch.matmul(tensor, tensor.T, out=y3)

print(f"matrix multiplication: \n {mat_mult} \n")


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
dot_mult = torch.mul(tensor, tensor, out=z3)

print(f"dot multiplication: \n {dot_mult} \n")

matrix multiplication: 
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]]) 

dot multiplication: 
 tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 



### Tensores de un solo elemento

Se puede tener un tensor de un solo elemento, en este caso, si se quiere convertir a número se puede usar el método `item()`

In [14]:
agg = tensor.sum()
print(f"Tensor de un solo elemento: {agg}, su dimensión es {agg.shape}")

agg_item = agg.item()
print(f"Tensor convertido a número: {agg_item}, es de tipo {type(agg_item)}")

Tensor de un solo elemento: 12.0, su dimensión es torch.Size([])
Tensor convertido a número: 12.0, es de tipo <class 'float'>


### Operaciones in situ

Son operaciones que se realizan sobre el propio elemento, se indican añadiendo un guión bajo `_` al final de la operación. Por ejemplo `x.copy_()` o `x.t_()` modificarán `x`

In [15]:
print(tensor, "\n")
tensor.add_(5)
print(tensor)

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]]) 

tensor([[6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.],
        [6., 5., 6., 6.]])


> Nota: Las operaciones in situ ahorran algo de memoria, pero pueden resultar problemáticas al calcular derivadas debido a una pérdida inmediata del historial. Por tanto, se desaconseja su uso.